This notebook should analyse bell inequalities that are found for the finite efficiency PR box. The bell inequalities
considered here should equalize at the threshold efficiency of the PR box.

We want to find out if there is a structure behind these bell inequalities.

In [7]:
import numpy as np
from linearbell.utils import get_configs, get_deterministic_behaviors, check_equiv_bell

ma = 2
mb = 2
n = 3

file = '../data/vertex_enum_pr_box_det_decomp/{}{}{}{}.gz'.format(ma,mb,n,n)
all_inequalities = np.loadtxt(file)

configs = get_configs(range(ma), range(mb), range(n), range(n))

dets = get_deterministic_behaviors(range(ma), range(mb), range(n))
p_origin = np.sum(dets, axis=0) / dets.shape[0]



For later testing define the pr boxes and shift their origins.

In [8]:
from linearbell.utils import general_pr_box_extended
eta = 4 / (4 + ma)
pr_box= [general_pr_box_extended(a, b, x, y, eta, range(2)) for (a, b, x, y) in configs]
pr_box = np.array(pr_box) - p_origin

epsilon = 0.01
pr_box_high = [general_pr_box_extended(a, b, x, y, eta + epsilon, range(2)) for (a, b, x, y) in configs]
pr_box_high = np.array(pr_box_high) - p_origin

pr_box_low= [general_pr_box_extended(a, b, x, y, eta - epsilon, range(2)) for (a, b, x, y) in configs]
pr_box_low = np.array(pr_box_low) - p_origin


As we have loaded all bell inequalities, we have to check how many bell inequalities are of the same class and if actually
all bell inequalities fulfill the conditions we have.

In [9]:
correct_inequalities = []
for b in all_inequalities:
    try:
        assert np.abs(pr_box @ b - 1) < 1e-4, 'Threshold eff PR box is not equalizing'
        assert pr_box_high @ b > 1.0, 'High efficiency Pr Box does not violate: PR * b = {}'.format(pr_box_high @ b)
        assert pr_box_low @ b < 1.0, 'Low efficiency PR box is not local: PR * b = {}'.format(pr_box_low @ b)

        correct_inequalities.append(b)
    except AssertionError as e:
        print(e)
correct_inequalities = np.array(correct_inequalities)

High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0
High efficiency Pr Box does not violate: PR * b = 1.0


Check how many classes of bell inequalities there are.

In [10]:
from linearbell.utils import affine_transform_bell
file_relabels = '../data/relabels_dets/{}{}{}{}.gz'.format(ma, mb, n, n)
relabels_dets = np.loadtxt(file_relabels, dtype=float).astype(int)
# shift the inequalities
correct_inequalities = affine_transform_bell(correct_inequalities, dets)

# find the number of classes
del_ineq = []
for i in range(correct_inequalities.shape[0]):
    # if this facet can already be deleted -> continue
    if i in del_ineq: continue
    for j in range(i + 1, correct_inequalities.shape[0]):
        # if this facet can already be deleted -> continue
        if j in del_ineq: continue
        # check if the two facets are equivalent
        if check_equiv_bell(correct_inequalities[i], correct_inequalities[j], relabels_dets, dets, tol=1e-4):
            del_ineq.append(j)
# store new facets
classes_inequalities = np.delete(correct_inequalities, del_ineq, axis=0)

As we can see not all of the bell inequalities we find are actually bell inequalities. Maybe we also have
to do a check for facets.

We have now the bell inequalities and want to display them in matrix form. Currently they are just in vector
form and the configuration of inputs / outputs at each entry is stored in the corresponding *configXXXX* variable.
Therefore define a function that returns the configurations in matrix form and a function that transforms vector form
bell inequalities to matrix form.

In [11]:
from itertools import product
def get_configs_mat(inputs_a, inputs_b, outputs_a, outputs_b):
    """ Returns the configurations in a matrix form """
    configs = []
    for y,b in product(inputs_b, outputs_b):
        c = []
        for x, a in product(inputs_a, outputs_a):
            c.append((a,b,x,y))
        configs.append(c)
    return configs

def transform_vec_to_mat(configs_vec, configs_mat, vec):
    """ Transforms a vector to matrix representation """
    mat = np.zeros((len(configs_mat), len(configs_mat[0])))
    for l,c in enumerate(configs_vec):
        for i in range(len(configs_mat)):
            try:
                j = configs_mat[i].index(c)
                # if not fails -> set the entry in matrix
                mat[i,j] = vec[l]
            except ValueError as e:
                pass
    return mat



Now use these functions to reshape the vectors of bell inequalities

In [12]:
configs_mat = get_configs_mat(range(ma), range(mb), range(n), range(n))

mat_inequalities = []
for b in correct_inequalities:
    m = transform_vec_to_mat(configs, configs_mat, b)
    mat_inequalities.append(m)

mat = mat_inequalities[0]